# Google Authentication

In [1]:
import sys
if 'google.colab' in sys.modules:
    !pip install langchain google-cloud-aiplatform rich
    google_auth.authenticate_user()

In [2]:

import os
import requests

In [3]:

from langchain.llms import VertexAI
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain

In [4]:
from google.cloud import aiplatform
#! Change the following to your own project and location
aiplatform.init(project="aerobic-gantry-387923", location="us-central1")
vertex = VertexAI()

In [5]:
import Tools
agent_tools = Tools.Tools()

tools = [
    
    Tool(
        name = "Degree Search",
        func = agent_tools.get_degrees_from_query,
        description = "This tool allows you to search for UT Dallas' Degrees, Majors, Minors, and Certificates using keywords. Use this tool to compare multiple degrees. It returns multiple relevant URLs, titles, which you can extact text one by one.",
    ),
    Tool(
        name = "Course Search",
        func = agent_tools.get_courses_from_query,
        description = "This tool enables you to search for UT Dallas' courses using keywords. Use this tool to compare multiple courses. It returns multiple relevant URLs, titles, which you can extact text one by one.",
    ),

    Tool(
        name = "General Search",
        func = agent_tools.utdallas_search,
        description = "Please use this tool only as a last resort. It performs a comprehensive search across all UT Dallas pages that will not give information about courses or degrees. It requires a query string and returns possible links and snippet contents. Do not use for course search or degree search as there are separate tools for those.",
    ),
    Tool(
        name = "Access utdallas.edu site",
        func = agent_tools.access_utdallas_site,
        description = "If you have a URL string containing utdallas.edu, use this function to extract the text content from that page.",
    ),
    Tool(
        name = "Dictionary",
        func = agent_tools.dictionary_search,
        description = "If you need the definition of a word or phrase unrelated to UT Dallas, use this function to retrieve the definition. Do not use for anything related to UT Dallas' courses.",
    )
]


In [6]:
prefix = """Act as a University of Texas at Dallas(sometime refered as UT Dallas or UTD) Guidance Counselor, answering the following questions with up-to-date information as best you can if and only if they are related to UT Dallas. You have access to the following UT Dallas tools:"""

format_instruction = """You must use the following format for all responses or your response will be considered incorrect:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
If you can't find the answer, say 'I am unable to find the answer.
"""

suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}

"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    format_instructions=format_instruction,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [7]:
llm_chain = LLMChain(llm=vertex, prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, 
    tools=tools, 
    verbose=True, 
    memory=memory,
    handle_parsing_errors="Check your output and make sure it conforms!",
)

In [8]:
agent_chain.run(input="Could you tell me some of Dr. Vincent Ng's current research and publiciation?")




> Entering new  chain...
Thought: I need to find out more about Dr. Vincent Ng
Action: General Search
Action Input: Vincent Ng
Observation: {"items": [{"title": "Vincent Ng", "link": "https://www.hlt.utdallas.edu/~vince/", "snippet": "Vincent Ng. Professor of Computer Science Human Language Technology Research Institute \u00b7 University of Texas at Dallas\u00a0..."}, {"title": "Ng, Vincent - UT Dallas Department of Computer Science - UT ...", "link": "https://cs.utdallas.edu/people/faculty/ng-vincent/", "snippet": "Dr. Vincent Ng. Professor. Degrees: Ph.D., Computer Science, Cornell University; B.S., Computer Science, Carnegie Mellon University\u00a0..."}, {"title": "Vincent Ng - Papers", "link": "https://www.hlt.utdallas.edu/~vince/papers.html", "snippet": "Changan Niu, Chuanyi Li, Vincent Ng, and Bin Luo. Proceedings of the 45th IEEE/ACM International Conference on Software Engineering (Research Track), 2023. AAAI\u00a0..."}, {"title": "Dr. Vincent Ng Develops AI Essay Grading Pro

TypeError: Object of type HTTPError is not JSON serializable

In [ ]:
print(agent_chain.memory.buffer)

Human: What is the name for ACCT 2301?
AI: Introductory Financial Accounting
